In [ ]:
import pandas as pd

In [ ]:
import pandas as pd

df1 = pd.read_csv('scc_results.csv')
p = df1['p_value']
out = df1['OTU']
df2 = pd.read_excel('otu.xlsx')
cui = list(df2['feature'])
new_df = pd.DataFrame(columns=df2.columns)

for a in range(len(p)):
    if p[a] < 0.05:
        new_row = df2.loc[cui.index(out[a])]
        new_df = pd.concat([new_df, pd.DataFrame(new_row).T], ignore_index=True)

new_df.to_csv('fit1.csv',index=False)

In [ ]:
df = pd.read_csv('fit1.csv')
df_transposed = df.transpose()
df_transposed.to_csv('newfit1.csv')


In [ ]:
n = pd.read_csv('newfit1.csv')
n['label'] = ''
g = pd.read_csv('group.csv')
gs = list(g['Sample'])
gg = list(g['Group'])
s = list(n['Sample'])
for b in range(len(s)):
    i = gs.index(s[b])
    n.loc[b,'label'] = gg[i]

In [ ]:
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel
n.to_csv('lfit1.csv',index=False)
# 创建示例数据
X = n.iloc[:, 1:51]  # 特征
y = n.iloc[:, 51]  # 标签

# 创建并拟合 Lasso 回归模型
lasso = Lasso(alpha=0.0005)  # 调整 alpha 参数来控制正则化强度
lasso.fit(X, y)

sfm = SelectFromModel(lasso)
X_selected = sfm.transform(X)

# 获取选择的特征索引
selected_feature_indices = sfm.get_support(indices=True)

selected_features = X.iloc[:, selected_feature_indices]
selected_features['label'] = y.values  # 将标签数据转换为数组形式
# 获取选择的特征的系数
selected_feature_contributions = lasso.coef_[selected_feature_indices]
selected_features.to_csv('feature.csv')
# 打印选择的特征的贡献度
print("Selected Feature Contributions:")
for i, idx in enumerate(selected_feature_indices):
    print(f"Feature {X.columns[idx]}: {selected_feature_contributions[i]}")

In [ ]:
import pandas as pd
f = pd.read_csv('/mnt/md0/Public/T3_T4/lasso/feature.csv')
fea = f.iloc[0:,1:-1]
feature_ = fea.astype("float").values
la = f.loc[0:,'label']
target_ = la.values